In [5]:
import pandas as pd
import BackEND ## my own file
from transformers import BertForSequenceClassification
from transformers import AdamW
import torch
from transformers import BertTokenizer
from torch.utils.data import DataLoader
import numpy as np

In [75]:
data=pd.read_csv("D:\PRojects\IPC\model\Train.csv")

In [76]:
texts_arr = data['Offense'].to_list()
labels = data['Section'].to_list()

In [77]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = BackEND.TextDataset(texts_arr, labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


c:\Users\ayush\miniconda3\envs\ipc\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [79]:
device

device(type='cuda')

In [82]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=444).to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in (train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}") 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch [1/30], Loss: 6.0712
Epoch [2/30], Loss: 5.7409
Epoch [3/30], Loss: 5.4061
Epoch [4/30], Loss: 5.1073
Epoch [5/30], Loss: 4.8399
Epoch [6/30], Loss: 4.5894
Epoch [7/30], Loss: 4.3555
Epoch [8/30], Loss: 4.1327
Epoch [9/30], Loss: 3.9051
Epoch [10/30], Loss: 3.6893
Epoch [11/30], Loss: 3.4807
Epoch [12/30], Loss: 3.2671
Epoch [13/30], Loss: 3.0718
Epoch [14/30], Loss: 2.8652
Epoch [15/30], Loss: 2.6728
Epoch [16/30], Loss: 2.4791
Epoch [17/30], Loss: 2.2943
Epoch [18/30], Loss: 2.1129
Epoch [19/30], Loss: 1.9377
Epoch [20/30], Loss: 1.7740
Epoch [21/30], Loss: 1.6195
Epoch [22/30], Loss: 1.4704
Epoch [23/30], Loss: 1.3262
Epoch [24/30], Loss: 1.1989
Epoch [25/30], Loss: 1.0721
Epoch [26/30], Loss: 0.9669
Epoch [27/30], Loss: 0.8670
Epoch [28/30], Loss: 0.7741
Epoch [29/30], Loss: 0.6886
Epoch [30/30], Loss: 0.6144


In [16]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [97]:
torch.save(model,'model.pth') ##torch format

In [ ]:
dum1=torch.randn(8,128).long().to(device)
dum2=torch.randn(8,128).long().to(device)
torch.onnx.export(model,(dum1,dum2),"model.onnx")  ## onnx format

Using it for prediction 

In [86]:
test=["wears any garb or carries any token resembling any garb or token used by such a soldier"]
lab=[0]

In [87]:
train_dataset = BackEND.TextDataset(test, lab, tokenizer)

In [88]:
train_dataset=train_dataset[0]

In [68]:
input_ids = train_dataset['input_ids'].unsqueeze(0).to(device)
attention_mask = train_dataset['attention_mask'].unsqueeze(0).to(device)

In [69]:
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits  # Raw model predictions

predicted_label = torch.argmax(logits, dim=-1).item()

In [71]:
import json

In [72]:
with open("D:\PRojects\IPC\Dataset_AUG\Mapping.json", 'r') as file:
    data_dict = json.load(file)


In [74]:
print(data_dict[str(predicted_label)])

IPC_140
